In [139]:
input_file = 'Day07Input.txt'

# Problem 1

## Helper Classes & Functions

In [140]:
class File:
    def __init__(self, name, size):
        self.name = name
        self.size = int(size)

In [141]:
class Directory:
    def __init__(self, name):
        self.name = name
        self.contents = []
        self.parent = None
            
    def add_subdir(self, subdir_name):
        self.contents.append(Directory(subdir_name))
        self.contents[-1].parent = self
        self.contents.sort(key=lambda x: x.name)
    
    def add_file(self, file_name, file_size):
        self.contents.append(File(file_name, file_size))
        self.contents.sort(key=lambda x: x.name)
    
    def get_size(self):
        size = 0
        for item in self.contents:
            if isinstance(item, File):
                size += item.size
            else:
                size += item.get_size()
        
        return size
    
    def __str__(self, level=0):
        if self.name == '/':
            s = '  ' * level + '- ' + self.name + '\n'
        else:
            s = '  ' * level + '- ' + self.name + '/\n'
            
        for item in self.contents:
            if isinstance(item, File):
                s += '  ' * (level + 1) + '- ' + f'{item.name} ({item.size})' + '\n'
            else:
                s += item.__str__(level + 1)
        
        return s

In [142]:
def parse_input(filepath):
    with open(filepath) as f:
        lines = f.readlines()

    for i in range(len(lines)):
        lines[i] = lines[i].strip()
    
    root = Directory('/')
    cwd = root
    line_num = 1
    while line_num < len(lines):
        line = lines[line_num]

        if line.startswith('$'):
            command = line[2:]

            if command == 'ls':
                i = line_num + 1
                while i < len(lines) and not lines[i].startswith('$'):
                    if lines[i].startswith('dir'):
                        cwd.add_subdir(lines[i].split(' ')[1])
                    else:
                        size, name = lines[i].split(' ')
                        cwd.add_file(name, size)
                    i += 1
                line_num = i

            elif command.startswith('cd'):
                dirname = command[3:]
                if dirname == '..':
                    cwd = cwd.parent
                else:
                    for item in cwd.contents:
                        if isinstance(item, Directory) and item.name == dirname:
                            cwd = item
                            break
                line_num += 1
    return root

## Create Filesystem

## Calculate Size

In [143]:
root = parse_input(input_file)

size = 0
dirs = [root]
for dir_ in dirs:
    dir_size = dir_.get_size()
    if dir_size <= 100_000:
        size += dir_size
    # print(f'{dir_.name}: {dir_.get_size()}')

    dirs.extend(item for item in dir_.contents if isinstance(item, Directory))
print(size)

1454188


# Problem 2

In [144]:
TOTAL_SPACE = 70_000_000
DESIRED_SPACE = 30_000_000

current_usage = root.get_size()
print(current_usage)

43837783


In [148]:
dirs = [item for item in root.contents if isinstance(item, Directory)]

options = []
for dir_ in dirs:
    dir_size = dir_.get_size()
    if current_usage - dir_size + DESIRED_SPACE <= TOTAL_SPACE:
        options.append((dir_.name, dir_size))

    dirs.extend(item for item in dir_.contents if isinstance(item, Directory))

print(min(options, key=lambda x: x[1]))

('wvq', 4183246)
